In [1]:
import os
from PIL import Image
import numpy as np

OPENSLIDE_PATH=r'C:\Program Files\openslide-win64-20230414\bin'
os.environ['PATH'] = OPENSLIDE_PATH + ";" + os.environ['PATH']

if hasattr(os, 'add_dll_directory'):
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

from openslide import OpenSlide


In [10]:
import os
from PIL import Image
import numpy as np

# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# print(tf.__version__)
OPENSLIDE_PATH= r'C:\Program Files\openslide-win64-20230414\bin'
os.environ['PATH'] = OPENSLIDE_PATH + ";" + os.environ['PATH']

if hasattr(os, 'add_dll_directory'):
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

from openslide import OpenSlide

In [11]:
wsisrc = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\Venous Invasion\PDAC-FS-3'
tenxsrc = os.path.join(wsisrc, '10X')
twosrc = os.path.join(wsisrc, '1X')
if not os.path.exists(tenxsrc):
    os.mkdir(tenxsrc)
if not os.path.exists(twosrc):
    os.mkdir(twosrc)

In [14]:
wsis = [_ for _ in os.listdir(wsisrc) if _.endswith('ndpi')]
wsis2 = [os.path.splitext(_)[0] for _ in wsis]
tenx = [_ for _ in os.listdir(tenxsrc) if _.endswith('tif')]
tenx2 = [os.path.splitext(_)[0] for _ in tenx]
missingimages = [_ for _ in wsis2 if _ not in tenx2]

In [15]:
for idx,missingimage in enumerate(missingimages):
    print(idx,'/',len(missingimages),' processing: ',missingimage)
    wsi = OpenSlide(os.path.join(wsisrc,missingimage+'.ndpi'))
    rsf = 1
    svs_img = wsi.read_region(location=(0,0),level=0,size=wsi.level_dimensions[0]).convert('RGB')
    resize_factorx = rsf/float(wsi.properties['openslide.mpp-x']) #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf/float(wsi.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)),int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST) #overwrite svs_img to save memory
    svs_img.save(os.path.join(tenxsrc,missingimage+'.tif'),resolution=1,resolution_unit=1,quality=100,compression=None)

    rsf = 8
    # for diabete study, rsf = 5; for vi study, rsf = 8
    resize_factorx = resize_factorx*rsf #8um = 1.25x # 5um = 2x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = resize_factory*rsf
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0]/resize_factorx)),int(np.ceil(wsi.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img.save(os.path.join(twosrc,missingimage+'.tif'),resolution=2_5,resolution_unit=1,quality=100,compression=None)

0 / 1  processing:  PDAC_FS_3_466
